Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (18724, 28, 28) (18724,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
    # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
    labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (18724, 784) (18724, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compue the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [49]:
batch_size = 100
n_hidden = 1024
L2_weight = 0.5e-3


def forward(tf_X):
    """
    assert tf.shape(tf_X)[1] == image_size*image_size,\
        "Training data not of correct shape. Each input should be of shape: %s" % (image_size*image_size)
    """
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([image_size*image_size, n_hidden]), name="weights")
        biases = tf.Variable(tf.zeros([n_hidden]), name="biases")
        z01 = tf.matmul(tf_X, weights) + biases
        hidden1 = tf.nn.relu(z01)
        l2_reg_01 = tf.nn.l2_loss(weights)
    with tf.name_scope('z12'):
        weights = tf.Variable(tf.truncated_normal([n_hidden, num_labels]), name="weights")
        biases = tf.Variable(tf.zeros([num_labels]), name="biases")
        z12 = tf.matmul(hidden1, weights) + biases
        l2_reg_12 = tf.nn.l2_loss(weights)
    return z12, l2_reg_01+l2_reg_12

# Define loss
def get_loss(z12, l2_loss, tf_Y):
    """
    assert tf.shape(tf_X)[1] == image_size*image_size,\
        "Training data not of correct shape. got %s require %s" % (tf.shape(tf_X)[1], image_size*image_size)
    assert tf.shape(tf_Y)[1] == num_labels,\
        "Training data not of correct shape. got %s require %s" % (tf.shape(tf_Y)[1], num_labels)
    """
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(z12, tf_training_labels))
    total_loss = loss #+ L2_weight*l2_loss
    return total_loss

# Define the network graph
graph = tf.Graph()
with graph.as_default():
    #tf_training_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    #tf_training_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_training_dataset = tf.placeholder(tf.float32) # Should have shape (batch_size, image_size*image_size)
    tf_training_labels = tf.placeholder(tf.float32) # Should have shape (batch_size, num_labels)
    
    z12, l2_loss = forward(tf_training_dataset)
    total_loss = get_loss(z12, l2_loss, tf_training_labels)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(total_loss)
    

In [57]:
# train the model
num_steps = 3001
batch_size = 100
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized, using batch size: %s" % batch_size
    for step in xrange(num_steps):
        idx = np.random.randint(train_dataset.shape[0], size=batch_size)
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[idx]
        batch_labels = train_labels[idx]
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_training_dataset : batch_data, tf_training_labels : batch_labels}
        _, l, predictions = session.run([optimizer, total_loss, z12], feed_dict=feed_dict)
        if (step % 500 == 0):
            #batch_size += 100
            print "Updated batch size: %s" % batch_size
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            predictions = session.run(z12, feed_dict={tf_training_dataset: valid_dataset})
            print "Validation accuracy: %.1f%%" % accuracy(predictions, valid_labels)
    predictions = session.run(z12, feed_dict={tf_training_dataset: test_dataset})
    print "Test accuracy: %.1f%%" % accuracy(predictions, test_labels)
        
    

Initialized, using batch size: 100
Updated batch size: 100
Minibatch loss at step 0 : 278.257
Minibatch accuracy: 13.0%
Validation accuracy: 32.8%
Updated batch size: 100
Minibatch loss at step 500 : 15.5628
Minibatch accuracy: 87.0%
Validation accuracy: 77.1%
Updated batch size: 100
Minibatch loss at step 1000 : 30.539
Minibatch accuracy: 78.0%
Validation accuracy: 77.2%
Updated batch size: 100
Minibatch loss at step 1500 : 15.8107
Minibatch accuracy: 82.0%
Validation accuracy: 78.5%
Updated batch size: 100
Minibatch loss at step 2000 : 12.5125
Minibatch accuracy: 77.0%
Validation accuracy: 79.4%
Updated batch size: 100
Minibatch loss at step 2500 : 6.22763
Minibatch accuracy: 83.0%
Validation accuracy: 80.6%
Updated batch size: 100
Minibatch loss at step 3000 : 4.98556
Minibatch accuracy: 74.0%
Validation accuracy: 80.7%
Test accuracy: 87.6%


In [58]:
train_dataset.shape

(200000, 784)

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [59]:
# Overfitting using very small subset of data
num_steps = 3001
batch_size = 100
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized, using batch size: %s" % batch_size
    for step in xrange(num_steps):
        idx = np.random.randint(train_dataset.shape[0]/100, size=batch_size)
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[idx]
        batch_labels = train_labels[idx]
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_training_dataset : batch_data, tf_training_labels : batch_labels}
        _, l, predictions = session.run([optimizer, total_loss, z12], feed_dict=feed_dict)
        if (step % 500 == 0):
            #batch_size += 100
            print "Updated batch size: %s" % batch_size
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            predictions = session.run(z12, feed_dict={tf_training_dataset: valid_dataset})
            print "Validation accuracy: %.1f%%" % accuracy(predictions, valid_labels)
    predictions = session.run(z12, feed_dict={tf_training_dataset: test_dataset})
    print "Test accuracy: %.1f%%" % accuracy(predictions, test_labels)
        
    

Initialized, using batch size: 100
Updated batch size: 100
Minibatch loss at step 0 : 343.88
Minibatch accuracy: 17.0%
Validation accuracy: 22.1%
Updated batch size: 100
Minibatch loss at step 500 : 0.151362
Minibatch accuracy: 98.0%
Validation accuracy: 78.4%
Updated batch size: 100
Minibatch loss at step 1000 : 0.0158547
Minibatch accuracy: 99.0%
Validation accuracy: 78.5%
Updated batch size: 100
Minibatch loss at step 1500 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Updated batch size: 100
Minibatch loss at step 2000 : 2.07879e-06
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Updated batch size: 100
Minibatch loss at step 2500 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Updated batch size: 100
Minibatch loss at step 3000 : 0.0
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Test accuracy: 86.5%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [67]:
batch_size = 100
n_hidden = 1024
L2_weight = 0.5e-3

def forward(tf_X, dropout_p):
    """
    assert tf.shape(tf_X)[1] == image_size*image_size,\
        "Training data not of correct shape. Each input should be of shape: %s" % (image_size*image_size)
    """
    with tf.name_scope('hidden1'):
        weights = tf.Variable(tf.truncated_normal([image_size*image_size, n_hidden]), name="weights")
        biases = tf.Variable(tf.zeros([n_hidden]), name="biases")
        z01 = tf.matmul(tf_X, tf.nn.dropout(weights, dropout_p)) + biases
        hidden1 = tf.nn.relu(z01)
        l2_reg_01 = tf.nn.l2_loss(weights)
    with tf.name_scope('z12'):
        weights = tf.Variable(tf.truncated_normal([n_hidden, num_labels]), name="weights")
        biases = tf.Variable(tf.zeros([num_labels]), name="biases")
        z12 = tf.matmul(hidden1, weights) + biases # NEVER USE DROPOUT ON LAST LAYER
        l2_reg_12 = tf.nn.l2_loss(weights)
    return z12, l2_reg_01+l2_reg_12

# Define loss
def get_loss(z12, l2_loss, tf_Y):
    """
    assert tf.shape(tf_X)[1] == image_size*image_size,\
        "Training data not of correct shape. got %s require %s" % (tf.shape(tf_X)[1], image_size*image_size)
    assert tf.shape(tf_Y)[1] == num_labels,\
        "Training data not of correct shape. got %s require %s" % (tf.shape(tf_Y)[1], num_labels)
    """
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(z12, tf_training_labels))
    total_loss = loss #+ L2_weight*l2_loss
    return total_loss

# Define the network graph
tf.python.framework.ops.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    #tf_training_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    #tf_training_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_training_dataset = tf.placeholder(tf.float32) # Should have shape (batch_size, image_size*image_size)
    tf_training_labels = tf.placeholder(tf.float32) # Should have shape (batch_size, num_labels)
    dropout_p = tf.placeholder(tf.float32)
    
    z12, l2_loss = forward(tf_training_dataset, dropout_p)
    total_loss = get_loss(z12, l2_loss, tf_training_labels)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(total_loss)

In [68]:
# train the model
num_steps = 3001
batch_size = 100
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized, using batch size: %s" % batch_size
    for step in xrange(num_steps):
        idx = np.random.randint(train_dataset.shape[0], size=batch_size)
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[idx]
        batch_labels = train_labels[idx]
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_training_dataset : batch_data, tf_training_labels : batch_labels, dropout_p: 0.5}
        _, l = session.run([optimizer, total_loss], feed_dict=feed_dict)
        predictions = session.run(z12, feed_dict={tf_training_dataset: batch_data, dropout_p: 1})
        if (step % 500 == 0):
            #batch_size += 100
            print "Updated batch size: %s" % batch_size
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            predictions = session.run(z12, feed_dict={tf_training_dataset: valid_dataset, dropout_p: 1})
            print "Validation accuracy: %.1f%%" % accuracy(predictions, valid_labels)
    predictions = session.run(z12, feed_dict={tf_training_dataset: test_dataset, dropout_p: 1})
    print "Test accuracy: %.1f%%" % accuracy(predictions, test_labels)
        

        

Initialized, using batch size: 100
Updated batch size: 100
Minibatch loss at step 0 : 377.01
Minibatch accuracy: 32.0%
Validation accuracy: 19.6%
Updated batch size: 100
Minibatch loss at step 500 : 67.7987
Minibatch accuracy: 87.0%
Validation accuracy: 76.5%
Updated batch size: 100
Minibatch loss at step 1000 : 36.1366
Minibatch accuracy: 84.0%
Validation accuracy: 78.5%
Updated batch size: 100
Minibatch loss at step 1500 : 31.1877
Minibatch accuracy: 83.0%
Validation accuracy: 78.9%
Updated batch size: 100
Minibatch loss at step 2000 : 16.6507
Minibatch accuracy: 85.0%
Validation accuracy: 78.4%
Updated batch size: 100
Minibatch loss at step 2500 : 31.9195
Minibatch accuracy: 73.0%
Validation accuracy: 76.6%
Updated batch size: 100
Minibatch loss at step 3000 : 50.2246
Minibatch accuracy: 81.0%
Validation accuracy: 76.9%
Test accuracy: 84.7%


In [69]:
# train the model using smaller sample resulting in overfitting
num_steps = 3001
batch_size = 100
with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print "Initialized, using batch size: %s" % batch_size
    for step in xrange(num_steps):
        idx = np.random.randint(train_dataset[:2000].shape[0], size=batch_size)
        #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_dataset[:2000][idx]
        batch_labels = train_labels[:2000][idx]
        #batch_data = train_dataset[offset:(offset + batch_size), :]
        #batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_training_dataset : batch_data, tf_training_labels : batch_labels, dropout_p: 0.5}
        _, l = session.run([optimizer, total_loss], feed_dict=feed_dict)
        predictions = session.run(z12, feed_dict={tf_training_dataset: batch_data, dropout_p: 1})
        if (step % 500 == 0):
            #batch_size += 100
            print "Updated batch size: %s" % batch_size
            print "Minibatch loss at step", step, ":", l
            print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
            predictions = session.run(z12, feed_dict={tf_training_dataset: valid_dataset, dropout_p: 1})
            print "Validation accuracy: %.1f%%" % accuracy(predictions, valid_labels)
    predictions = session.run(z12, feed_dict={tf_training_dataset: test_dataset, dropout_p: 1})
    print "Test accuracy: %.1f%%" % accuracy(predictions, test_labels)
        

Initialized, using batch size: 100
Updated batch size: 100
Minibatch loss at step 0 : 478.766
Minibatch accuracy: 44.0%
Validation accuracy: 29.6%
Updated batch size: 100
Minibatch loss at step 500 : 2.97874
Minibatch accuracy: 98.0%
Validation accuracy: 80.2%
Updated batch size: 100
Minibatch loss at step 1000 : 6.14865
Minibatch accuracy: 97.0%
Validation accuracy: 79.6%
Updated batch size: 100
Minibatch loss at step 1500 : 3.33054
Minibatch accuracy: 100.0%
Validation accuracy: 81.0%
Updated batch size: 100
Minibatch loss at step 2000 : 0.352523
Minibatch accuracy: 100.0%
Validation accuracy: 80.6%
Updated batch size: 100
Minibatch loss at step 2500 : 7.5045
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Updated batch size: 100
Minibatch loss at step 3000 : 0.0056987
Minibatch accuracy: 100.0%
Validation accuracy: 80.7%
Test accuracy: 88.0%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---
